In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
import sys
from pathlib import Path
print(Path.cwd())


/Users/vaishnukanna/Documents/jegadeesh-titman-momentum/results


In [3]:
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [4]:
from src.config import PRICES_PATH
print(PRICES_PATH, PRICES_PATH.exists())

/Users/vaishnukanna/Documents/jegadeesh-titman-momentum/data/raw/prices.csv True


In [5]:
from src.returns import load_prices, compute_monthly_returns
from src.momentum_strategy import get_winner_loser_masks, rank_stocks, get_J_returns
from src.portfolio import compute_portfolio_returns

In [6]:
prices = load_prices()
monthly_returns = compute_monthly_returns(prices)

In [ ]:
returns = {}
J = [3,6,9,12]
K = [3,6,9,12]
for j in J:
    for k in K:
        formation_returns = get_J_returns(j)
        deciles = rank_stocks(formation_returns)
        winners, losers = get_winner_loser_masks(deciles)
        ret = compute_portfolio_returns(monthly_returns,winners, losers, j,k,skip=0)["spread"]
        returns[f"J={j}_K={k}"] = ret
        volatility[f"J={j}_K={k}"] = ret.std()

### Newey west means and t-stat

In [9]:
#The mean of a time series is effectively the intercept in a regression model 
#where the only independent variable is a constant (a column of ones).
def nw_mean_tstat(returns, lags):
    r = returns.dropna()
    T = len(r)
    X = np.ones(T)
    model = sm.OLS(r.values, X)    #y=returns, X = 1, beta (coeff) will be the means, 
    #we need to adjust means for heteroscedasticity and serial correlation
    res = model.fit(cov_type='HAC', cov_kwds={'maxlags':lags})
    return res.params[0], res.tvalues[0]  #return mean and t-stat


In [10]:
means = pd.DataFrame(index = K, columns=J, dtype = float)
t_stats = pd.DataFrame(index = K, columns=J, dtype = float)


In [11]:
for j in J:
    for k in K:
        ret = returns[f"J={j}_K={k}"]
        mean, t_stat = nw_mean_tstat(ret, k-1)
        means.loc[k,j] = mean*100
        t_stats.loc[k,j] = t_stat

In [12]:
formatted_table = pd.DataFrame(
    index=means.index,
    columns=means.columns,
    dtype=str
)

for K in means.index:
    for J in means.columns:
        mean = means.loc[K, J]
        tstat = t_stats.loc[K, J]
        formatted_table.loc[K, J] = f"{mean:.2f} ({tstat:.2f})"

In [13]:
formatted_table.to_csv(PROJECT_ROOT/"results/table1.csv")